
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Get AWS ELB Unhealthy Instances using unSkript legos.</b>
</div>

<br>

<center><h2>Get AWS ELB Unhealthy Instances</h2></center>

# Steps Overview
    1) Get all AWS ELB unhealthy instances by using elb_name as input.
    2) Post the AWS ELB unhealthy instances to the slack channel.

Here we will use unSkript Get UnHealthy EC2 Instances for Classic ELB Lego. This lego takes elb_name as input. This inputs is used to findout all unHealthy EC2 instances for Classic ELB.

In [ ]:
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_get_unhealthy_instances(handle, elb_name: str):

    elbClient = handle.client('elb')
    res = elbClient.describe_instance_health(
        LoadBalancerName=elb_name,
    )

    unhealthy_instances = []
    for instance in res['InstanceStates']:
        if instance['State'] == "OutOfService":
            unhealthy_instances.append(instance)

    return unhealthy_instances


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "elb_name": "elb_name"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_get_unhealthy_instances, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

Here we will use unSkript Post Slack Message Lego. This lego takes channel: str and message: str as input. This inputs is used to post the message to the slack channel.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Unhealthy instances for elb:{elb_name} {elb_output}\\""
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

### Conclusion
In this Runbook, we demonstrated the use of unSkript's AWS and slack legos to perform AWS action and this runbook fetches the unHealthy EC2 instances for Classic ELB and posts to a slack channel. To view the full platform capabilities of unSkript please visit https://unskript.com